<a href="https://colab.research.google.com/github/matherdy/Ukraine-War-Reserach/blob/master/personal_folders/mark_heine/RussoUkraine_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np

In [3]:
data = yf.download(tickers='ICLN', period='1y', interval='1d',prepost= True)

[*********************100%***********************]  1 of 1 completed


In [4]:
def exponential_moving_average(data, window):
    ema = data['Close'].ewm(span=window, adjust=False).mean()
    return ema

In [5]:
def ema(data, window = 20):
    close = data["Close"]
    
    # Calculate EMA
    ema = close.ewm(span=window).mean()
    
    # Add EMA to the dataframe
    data['EMA'] = ema
    
    return data

ema(data)

,Open,High,Low,Close,Adj Close,Volume,EMA
Date,,,,,,,
2022-03-02,20.400000,20.420000,19.910000,20.180000,20.008181,5728100,20.180000
2022-03-03,19.950001,19.959999,19.430000,19.570000,19.403374,7243100,19.859750
2022-03-04,19.469999,19.670000,19.190001,19.420000,19.254652,6166800,19.698276
2022-03-07,20.020000,20.180000,19.840000,19.900000,19.730564,11986100,19.756511
2022-03-08,20.570000,21.610001,20.420000,21.270000,21.088902,17680400,20.122611
...,...,...,...,...,...,...,...
2023-02-23,19.490000,19.500000,19.049999,19.290001,19.290001,2556400,19.898225
2023-02-24,18.990000,19.030001,18.809999,18.930000,18.930000,2682200,19.806013
2023-02-27,19.160000,19.340000,19.049999,19.250000,19.250000,1969200,19.753059


In [6]:
import plotly.graph_objs as go

# Create traces for candlestick chart
candlestick = go.Candlestick(x=data.index, open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'], name='Candlestick')

# Create traces for EMA
ema20 = go.Scatter(x=data.index, y=data['Close'].ewm(span=20).mean(), line=dict(color='blue', width=1.5), name='20-day EMA')
ema50 = go.Scatter(x=data.index, y=data['Close'].ewm(span=50).mean(), line=dict(color='red', width=1.5), name='50-day EMA')

# Add traces to figure
fig = go.Figure()
fig.add_trace(candlestick)
fig.add_trace(ema20)
fig.add_trace(ema50)

# Add titles
fig.update_layout(title='ICLN live share price evolution with EMA',
                  yaxis_title='Stock Price (USD per Shares)')

# X-Axes
fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(buttons=list([
                     dict(count=15, label="15m", step="minute", stepmode="backward"),
                     dict(count=45, label="45m", step="minute", stepmode="backward"),
                     dict(count=1, label="HTD", step="hour", stepmode="todate"),
                     dict(count=3, label="3h", step="hour", stepmode="backward"),
                     dict(step="all")
                 ])))

fig.show()

In [7]:
data['EMA Signal'] = 0

# Calculate 20-day EMA
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()

# Assign 1 to "EMA Signal" column for buy signals and 0 for sell signals
data.loc[data['Close'] > data['EMA_20'], 'EMA Signal'] = 1
data.loc[data['Close'] < data['EMA_20'], 'EMA Signal'] = 0

In [8]:
test = data[data["EMA Signal"] !=0]

In [9]:
data['EMA_Signal'] = 0
data['EMA_Signal'] = np.where(data['EMA_20'] > data['Close'], 1, 0)  # buy signal
data['EMA_Signal'] = np.where(data['EMA_20'] < data['Close'], -1, data['EMA_Signal'])  # sell signal

In [10]:
fig.add_trace(go.Scatter(x=data.index[data["EMA_Signal"]==1], y = data["Close"][data["EMA_Signal"]==1], 
                         mode = "markers", marker_color = "darkgreen", marker_symbol = "arrow-up", marker_size = 10,
                        name = "Buy Signal")) #buy signal
fig.add_trace(go.Scatter(x=data.index[data["EMA_Signal"]==-1], y = data["Close"][data["EMA_Signal"]==-1], 
                         mode = "markers", marker_color = "darkred", marker_symbol = "arrow-down", marker_size = 10,
                        name = "Sell Signal")) #sell signal


In [11]:
def get_ema_returns(data):
    entrie_prices = []
    exit_prices = []
    entrie_dates = []
    exit_dates = []
    in_market = False
    for i in range(data.shape[0]):
        if (in_market == False) and (data.iloc[i]["EMA_Signal"] == 1):
            in_market = True
            entrie_prices.append((data.iloc[i]["Close"]))
            entrie_dates.append(data.iloc[i].name)
        if in_market == True and (data.iloc[i]["EMA_Signal"] == -1):
            in_market = False
            exit_prices.append((data.iloc[i]["Close"]))
            exit_dates.append(data.iloc[i].name)
    if len(entrie_prices) > len(exit_prices):
        exit_prices.append(data.iloc[-1]["Close"])
        exit_dates.append(data.iloc[-1].name)
    return sum(exit_prices)-sum(entrie_prices), entrie_dates, exit_dates

ema_returns, ema_entries, ema_exits = get_ema_returns(data)
print("EMA Trading Strategy Returns:", round(ema_returns, 2))


EMA Trading Strategy Returns: -0.85


In [12]:
ema_returns, ema_entries, ema_exits

(-0.8500022888183594,
 [Timestamp('2022-03-03 00:00:00'),
  Timestamp('2022-03-14 00:00:00'),
  Timestamp('2022-04-08 00:00:00'),
  Timestamp('2022-05-05 00:00:00'),
  Timestamp('2022-06-10 00:00:00'),
  Timestamp('2022-06-28 00:00:00'),
  Timestamp('2022-07-05 00:00:00'),
  Timestamp('2022-07-12 00:00:00'),
  Timestamp('2022-08-22 00:00:00'),
  Timestamp('2022-08-29 00:00:00'),
  Timestamp('2022-09-15 00:00:00'),
  Timestamp('2022-11-02 00:00:00'),
  Timestamp('2022-12-16 00:00:00'),
  Timestamp('2023-01-19 00:00:00'),
  Timestamp('2023-01-27 00:00:00'),
  Timestamp('2023-02-03 00:00:00')],
 [Timestamp('2022-03-08 00:00:00'),
  Timestamp('2022-03-16 00:00:00'),
  Timestamp('2022-05-04 00:00:00'),
  Timestamp('2022-05-19 00:00:00'),
  Timestamp('2022-06-23 00:00:00'),
  Timestamp('2022-07-01 00:00:00'),
  Timestamp('2022-07-07 00:00:00'),
  Timestamp('2022-07-19 00:00:00'),
  Timestamp('2022-08-24 00:00:00'),
  Timestamp('2022-09-07 00:00:00'),
  Timestamp('2022-10-26 00:00:00'),
  Tim